In [69]:
import pandas as pd
import numpy as np

np.random.seed(1349)

In [70]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

In [71]:
df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

In [72]:
df

,name,math,english,reading,classroom
0,Sally,78,84,73,B
1,Jane,77,79,98,B
2,Suzie,96,91,91,A
3,Billy,62,88,72,B
4,Ada,98,88,92,B
5,John,95,73,64,A
6,Thomas,87,80,82,A
7,Marie,99,85,94,A
8,Albert,91,77,87,B
9,Richard,84,61,94,A


## Indexing and Subsetting

Like the pandas Series object, the pandas DataFrame object supports both position- and label-based indexing using the indexing operator `[]`.

I will demonstrate concrete examples of indexing using the indexing operator `[]` alone and with the `.loc` and `.iloc` attributes below.

### `[]`

I can pass a list of columns from a DataFrame to the indexing operator (aka bracket notation) to return a subset of my original DataFrame.

In [73]:
df.name

0       Sally
1        Jane
2       Suzie
3       Billy
4         Ada
5        John
6      Thomas
7       Marie
8      Albert
9     Richard
10      Isaac
11       Alan
Name: name, dtype: object

In [ ]:
# problem: df.class


In [79]:
df['name']

0       Sally
1        Jane
2       Suzie
3       Billy
4         Ada
5        John
6      Thomas
7       Marie
8      Albert
9     Richard
10      Isaac
11       Alan
Name: name, dtype: object

In [80]:
df[['name']]

,name
0,Sally
1,Jane
2,Suzie
3,Billy
4,Ada
5,John
6,Thomas
7,Marie
8,Albert
9,Richard


In [81]:
df.columns

Index(['name', 'math', 'english', 'reading', 'classroom'], dtype='object')

In [83]:
df[['name', 'math']]

,name,math
0,Sally,78
1,Jane,77
2,Suzie,96
3,Billy,62
4,Ada,98
5,John,95
6,Thomas,87
7,Marie,99
8,Albert,91
9,Richard,84


In [ ]:
# I can choose a single column using bracket notation.



In [85]:
df[df.name.str.startswith('A')]

,name,math,english,reading,classroom
4,Ada,98,88,92,B
8,Albert,91,77,87,B
11,Alan,83,87,60,B


In [86]:
# I can pass a boolean Series to the indexing operator as a selector.
# names that start with a?
bools = df.name.str.startswith('A')

In [87]:
df[bools]

,name,math,english,reading,classroom
4,Ada,98,88,92,B
8,Albert,91,77,87,B
11,Alan,83,87,60,B


### `.loc`

We can use the `.loc` attribute to select specific rows AND columns by index label. The index label can be a number, but it can also be a string label. This method offers a lot of flexibility! **The .loc attribute's indexing is inclusive and uses an index label, not position.**

```python
df.loc[row_indexer, column_indexer]
```

In [88]:
somelist = [1,2,3,4,5,6,7]

In [92]:
somelist[:2]

[1, 2]

In [93]:
df.loc[:]

,name,math,english,reading,classroom
0,Sally,78,84,73,B
1,Jane,77,79,98,B
2,Suzie,96,91,91,A
3,Billy,62,88,72,B
4,Ada,98,88,92,B
5,John,95,73,64,A
6,Thomas,87,80,82,A
7,Marie,99,85,94,A
8,Albert,91,77,87,B
9,Richard,84,61,94,A


In [94]:
# Select all the rows and a subset of columns; notice the inclusive behavior of the indexing.

df.loc[:4]

,name,math,english,reading,classroom
0,Sally,78,84,73,B
1,Jane,77,79,98,B
2,Suzie,96,91,91,A
3,Billy,62,88,72,B
4,Ada,98,88,92,B


In [95]:
# I can use a boolean Series as a selector with .loc, too, but I can choose rows and columns.
# dimensionality: 0, 1: rows, columns
df.loc[:4, 'name':'english']


,name,math,english
0,Sally,78,84
1,Jane,77,79
2,Suzie,96,91
3,Billy,62,88
4,Ada,98,88


In [99]:
df.loc[:4, ['name','english']]

,name,english
0,Sally,84
1,Jane,79
2,Suzie,91
3,Billy,88
4,Ada,88


In [106]:
df.loc[bools]

,name,math,english,reading,classroom
4,Ada,98,88,92,B
8,Albert,91,77,87,B
11,Alan,83,87,60,B


### `.iloc`

We can use the `.iloc` attribute to select specific rows and colums by index position. `.iloc` does not accept a boolean Series as a selector like `.loc` does. **It takes in integers representing index position and is NOT inclusive.**

```python
df.iloc[row_indexer, column_indexer]
```

We can select rows by integer position:

In [102]:
# Notice the exclusive behavior of the indexing.

df.iloc[:3]

,name,math,english,reading,classroom
0,Sally,78,84,73,B
1,Jane,77,79,98,B
2,Suzie,96,91,91,A


We can also specify which columns we want to select:

In [103]:
df.iloc[:3, 1:4]

,math,english,reading
0,78,84,73
1,77,79,98
2,96,91,91


Here we select the first 3 rows (everything up to but not including the index of 3), and the second and third columns (starting from the index of 1 up to but not including the index of 3).

## Aggregating

### `.agg`

The `.agg` method lets us specify a way to aggregate a series of numerical values. We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [112]:
df.agg('min')

name         Ada
math          62
english       61
reading       60
classroom      A
dtype: object

In [113]:
df

,name,math,english,reading,classroom
0,Sally,78,84,73,B
1,Jane,77,79,98,B
2,Suzie,96,91,91,A
3,Billy,62,88,72,B
4,Ada,98,88,92,B
5,John,95,73,64,A
6,Thomas,87,80,82,A
7,Marie,99,85,94,A
8,Albert,91,77,87,B
9,Richard,84,61,94,A


In [111]:
df.agg(['min', 'max'])

,name,math,english,reading,classroom
min,Ada,62,61,60,A
max,Thomas,99,91,98,B


In [110]:
# I can use it on the entire df.
df[['math', 'english']].agg(['min', 'max'])

,math,english
min,62,61
max,99,91


In [109]:
df.describe()

,math,english,reading
count,12.000000,12.000000,12.000000
mean,85.583333,82.000000,83.333333
std,11.007917,8.696917,12.928709
min,62.000000,61.000000,60.000000
25%,77.750000,78.500000,72.750000
50%,85.500000,84.500000,89.000000
75%,95.250000,88.000000,93.250000
max,99.000000,91.000000,98.000000


In [117]:
# I can use it on a single column in a df.

df.reading.agg(['min', 'mean'])

min     60.000000
mean    83.333333
Name: reading, dtype: float64

In [118]:
# I can pass a list of functions to the .agg method.

df[['math', 'reading', 'english']].agg(['min','max','mean'])

,math,reading,english
min,62.000000,60.000000,61.0
max,99.000000,98.000000,91.0
mean,85.583333,83.333333,82.0


While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

### `.groupby`

The `.groupby` method is used to create a grouped object, which we can then apply an aggregation on. For example, if we wanted to know the highest math grade from each classroom:

In [122]:
df.groupby('classroom')

In [ ]:
# get our maximum grade for our 'math' column for classroom A and B respectively

In [123]:
df.groupby('classroom').math.max()

classroom
A    99
B    98
Name: math, dtype: int64

We can use `.agg` here to, to see multiple aggregations:

In [124]:
# structure:
# take your dataframe df
# group by classroom column
# use the math column --> referenced with .math (could use ['math'] instead)
# use a math method --> aggregation via .agg
# use these functions in agg: min, mean, max
groupby1 = df.groupby('classroom').math.agg(['min', 'mean', 'max'])

In [143]:
groupby1 = df[['math', 'reading','classroom']].groupby('classroom').agg(['min', 'mean', 'max'])

In [144]:
groupby1

math                reading               
           min       mean max     min       mean max
classroom                                           
A           77  89.666667  99      64  86.333333  94
B           62  81.500000  98      60  80.333333  98

In [129]:
groupby1.columns = ['math_min_grade', 'math_avg_grade', 'math_max_grade']
groupby1

,math_min_grade,math_avg_grade,math_max_grade
classroom,,,
A,77,89.666667,99
B,62,81.500000,98


We can group by multiple columns as well. To demonstrate, we'll create a boolean column named `passing_math`, then group by the combination of our new feature, `passing_math`, and the classroom and calculate the average reading grade and the number of individuals in each subgroup. 

Let's break this problem down and code it step-by-step.

#### `np.where`

First, we can create the new `passing_math` column using a handy NumPy function called `np.where`. It will allow us to base the new column values on whether the values in an existing column, `math`, meet a condition.

```python
np.where(condition, this_where_True, this_where_False)
```

In [131]:
df.math < 70

0     False
1     False
2     False
3      True
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
Name: math, dtype: bool

In [146]:
np.where(df.math < 70, 'failing', df.math + 7)

array(['85', '84', '103', 'failing', '105', '102', '94', '106', '98',
       '91', '84', '90'], dtype='<U21')

In [147]:
df.name.str.startswith('A')

0     False
1     False
2     False
3     False
4      True
5     False
6     False
7     False
8      True
9     False
10    False
11     True
Name: name, dtype: bool

In [152]:
df['a_name_student'] = np.where(df.name.str.startswith('A'), 'A student', np.NaN)

In [153]:
df

,name,math,english,reading,classroom,passing_math,a_name_student
0,Sally,78,84,73,B,passing,nan
1,Jane,77,79,98,B,passing,nan
2,Suzie,96,91,91,A,passing,nan
3,Billy,62,88,72,B,failing,nan
4,Ada,98,88,92,B,passing,A student
5,John,95,73,64,A,passing,nan
6,Thomas,87,80,82,A,passing,nan
7,Marie,99,85,94,A,passing,nan
8,Albert,91,77,87,B,passing,A student
9,Richard,84,61,94,A,passing,nan


Argument descriptors:
np.where(condition, thing if true, thing if false)
condition: math grade is less than 70 --> df.math < 70
output if True: 'failing'
output if False: 'passing'

In [132]:
# Create the new column based on an existing column.

df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')
df

,name,math,english,reading,classroom,passing_math
0,Sally,78,84,73,B,passing
1,Jane,77,79,98,B,passing
2,Suzie,96,91,91,A,passing
3,Billy,62,88,72,B,failing
4,Ada,98,88,92,B,passing
5,John,95,73,64,A,passing
6,Thomas,87,80,82,A,passing
7,Marie,99,85,94,A,passing
8,Albert,91,77,87,B,passing
9,Richard,84,61,94,A,passing


In [133]:
df['fakecol'] = df.math.apply(lambda x: 'failing' if x < 70 else 'passing')

In [139]:
df.drop(columns='fakecol', inplace=True)

Next, we will group by the `passing_math` and `classroom` columns and use the `.agg` method to calculate the average reading grade and the number of students.

In [154]:
grade_groups = df.groupby(['passing_math', 'classroom']).reading.agg(['mean', 'count'])
grade_groups

mean  count
passing_math classroom                  
failing      B          72.000000      1
passing      A          86.333333      6
             B          82.000000      5

In [155]:
# I can even clean up my columns to make my calculations clearer.

grade_groups.columns = ['avg_reading_grade', 'count_of_students']
grade_groups

avg_reading_grade  count_of_students
passing_math classroom                                      
failing      B                  72.000000                  1
passing      A                  86.333333                  6
             B                  82.000000                  5

**Takeaways:**

We can interpret this output as there being 2 students failing math in classroom A with an average reading grade of 87, 6 students passing math in classroom A with an average reading grade of 87.16, and 4 students passing math in classroom B with an average reading grade of 85.25.

#### `.transform`

The `.transform` method can be used to produce a series with the same length of the original dataframe where each value represents the aggregation from the subgroup resulting from the `.groupby`. 

For example, if we wanted to know the average math grade for each classroom and add this data back to our original dataframe:

In [158]:
df.assign(avg_math_score_by_classroom=df.groupby('classroom').math.transform('mean'))

,name,math,english,reading,classroom,passing_math,a_name_student,avg_math_score_by_classroom
0,Sally,78,84,73,B,passing,nan,81.500000
1,Jane,77,79,98,B,passing,nan,81.500000
2,Suzie,96,91,91,A,passing,nan,89.666667
3,Billy,62,88,72,B,failing,nan,81.500000
4,Ada,98,88,92,B,passing,A student,81.500000
5,John,95,73,64,A,passing,nan,89.666667
6,Thomas,87,80,82,A,passing,nan,89.666667
7,Marie,99,85,94,A,passing,nan,89.666667
8,Albert,91,77,87,B,passing,A student,81.500000
9,Richard,84,61,94,A,passing,nan,89.666667


#### `.describe`

Check out what I can do when I combine a `.groupby` with a `.describe`!

In [163]:
df.groupby('classroom').describe().T

classroom              A          B
math    count   6.000000   6.000000
        mean   89.666667  81.500000
        std     8.430105  12.469964
        min    77.000000  62.000000
        25%    84.750000  77.250000
        50%    91.000000  80.500000
        75%    95.750000  89.000000
        max    99.000000  98.000000
english count   6.000000   6.000000
        mean   80.166667  83.833333
        std    11.634718   4.792355
        min    61.000000  77.000000
        25%    74.750000  80.250000
        50%    82.500000  85.500000
        75%    89.500000  87.750000
        max    91.000000  88.000000
reading count   6.000000   6.000000
        mean   86.333333  80.333333
        std    11.843423  14.348054
        min    64.000000  60.000000
        25%    84.250000  72.250000
        50%    92.000000  80.000000
        75%    93.750000  90.750000
        max    94.000000  98.000000

## Merging and Joining

Pandas provides several ways to combine dataframes together. We will look at two of them below:

### `pd.concat`

This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis, row-wise axis=0 or column-wise axis=1.

```python
# For example, concat with a list of two DataFrames
pd.concat([df1, df2], axis=0)
```

- When your list contains at least one DataFrame, a DataFrame is returned.


- When concatenating only Series objects row-wise, axis=0, a Series is returned.


- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

```python
# Default is set to row-wise concatenation using an outer join.
pd.concat(objs, axis=0, join='outer')
```

When concatenating dataframes vertically, we basically are just adding more rows to an existing dataframe. In this case, the dataframes we are putting together should have the same column names[^1].

In [164]:
df1 = pd.DataFrame({'a': [1, 2, 3]})
df2 = pd.DataFrame({'a': [4, 5, 6]})

df1

,a
0,1
1,2
2,3


In [165]:
df2

,a
0,4
1,5
2,6


In [166]:
pd.concat([df1,df2])

,a
0,1
1,2
2,3
0,4
1,5
2,6


**Note** that the indices are preserved on the resulting dataframe; we could set the `ignore_index` parameter to `True` if we wanted these to be sequential.

In [167]:
concat_df1 = pd.concat([df1, df2], ignore_index=True)
concat_df1

,a
0,1
1,2
2,3
3,4
4,5
5,6


[^1]:
    We can concatenate dataframes with different column names, but generally this is not the behavior we want, as pandas will fill in a lot of null values into the resulting dataframe. The exception to this is if the dataframes are aligned on their index (i.e. the labels for each row), then we can provide the `axis=1` keyword argument to `pd.concat` to merge the dataframes horizontally.

In [168]:
concat_df2 = pd.DataFrame({'b': [1, 2, 3, 4, 5, 6]})
concat_df2

,b
0,1
1,2
2,3
3,4
4,5
5,6


In [171]:
pd.concat([concat_df1, concat_df2], axis=1)

,a,b
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6


In [ ]:
pd.concat([concat_df1, df1], axis=1)

### `.merge`

This method is similar to a SQL join. Here's a [cool read](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join) making a comparison between the two, if you're interested.

```python
# df.merge default settings for commonly used parameters.

left_df.merge(right_df, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)
```

How does changing the default argument of the `how` parameter change my resulting DataFrame?

`how` == Type of merge to be performed.

`how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.

`how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.

`how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

`how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [172]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [173]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


The `.merge` method will allow us to specify `left_on` and `right_on` to indicate the columns that are the keys used to merge the dataframes together. 

- In addition, the `how` keyword argument is used to define what type of JOIN we want to do; as we saw above, `inner` is the default setting. 

- For demonstration purposes, I'm also going to set the `indicator` parameter to `True`, which will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [176]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


Notice that we have duplicate column names in the resulting dataframe. By default, pandas will add a suffix of `_x` to any columns in the left dataframe that are duplicated, and `_y` to any columns in the right dataframe that are duplicated. I can clean up my columns if I want to; one way would be to use method chaining, which it demonstrated below:

In [181]:
# left_on = left table key
# right_on = right table key

# ON left.key = right.key


In [180]:
temp = (users.merge(roles, 
            left_on='role_id', 
            right_on='id')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)
temp

,id,employee,role_id,role
0,1,bob,1,admin
1,2,joe,2,author
2,3,sally,3,reviewer
3,4,adam,3,reviewer


In [179]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        6 non-null      float64
 1   employee  6 non-null      object 
 2   role_id   5 non-null      float64
 3   role      5 non-null      object 
dtypes: float64(2), object(2)
memory usage: 280.0+ bytes
